# 要約 
このJupyterノートブックは、Kaggleの「20の質問」コンペティションに参加するためのエージェント（ロボット）を作成する方法を示しています。ノートブックは、特に初心者向けに設計されており、ゲーム内で質問者、回答者、推測者の役割を果たすことができるモデルの実装に焦点を当てています。

### 取り組んでいる問題
このノートブックは、言語モデル（LLM）を用いて「20の質問」ゲームをプレイするエージェントを構築するという課題に取り組んでいます。ユーザーが選んだ単語を質問を通じて推測し、また、その質問に対して「はい」または「いいえ」で回答する必要があります。

### 使用している手法とライブラリ
1. **ライブラリ**:
   - `transformers`: Hugging FaceのTransformersライブラリを使用して、事前訓練された言語モデルをロードする。
   - `torch`: PyTorchを使用して、モデルの推論を行います。
   - `kaggle_secrets`: Kaggle環境でのシークレット管理に使用。

2. **モデル初期化**:
   - `AutoTokenizer` と `AutoModelForCausalLM` を利用して、モデルとトークナイザーを初期化。

3. **エージェントの設計**:
   - `Robot`クラスを定義し、質問者、回答者、推測者のそれぞれの役割を果たすメソッドを提供。
   - 質問は「はい」または「いいえ」で答える形式とし、適切なプロンプトを与えて生成する質問を制御。

4. **エージェントの実行**:
   - `agent`関数において、ゲームの状態に応じて適切なモード（質問、推測、回答）を設定し、ロボットが正しい応答を生成するように設計されている。

5. **ローカルテスト**:
   - 環境内でのシミュレーションとローカルテストが可能なコードが含まれています。

ノートブックは最後に、提出用のアーカイブファイルを作成するためのコードも含まれており、コンペティションにエージェントを提出する際の手順に従っています。

全体として、このノートブックは20の質問ゲームを効果的にプレイできるエージェントを実装するための明確な手順を提供しており、使用するライブラリや手法を詳細に解説しています。

---


# 用語概説 
以下に初心者がつまずきそうな専門用語の簡単な解説を示します。特に、実務を経験していないと馴染みのないものや、ノートブック特有のドメイン知識に焦点を当てました。

1. **LLM (Large Language Model)**:
   大規模言語モデルの略。大量のテキストデータを基に学習され、文脈を理解し、テキスト生成や質問応答などのタスクを遂行できるAIモデル。

2. **トークナイザー (Tokenizer)**:
   テキストをトークンと呼ばれる小さな単位（単語やサブワードなど）に分割するツール。モデルが入力を理解しやすくするために、テキストを数値に変換するプロセスの一部。

3. **CUDA (Compute Unified Device Architecture)**:
   NVIDIA社によって開発された並列コンピューティングアーキテクチャ。GPUを用いた高速な計算を可能にし、大規模なデータ処理や深層学習を効率的に行うことができる。

4. **メモリ効率 (Memory Efficiency)**:
   モデルが使用するメモリの効率的な管理を指す。特にGPUでの計算時に、メモリ使用量を抑える工夫が施されている。

5. **バイノーム (Binomial)**:
   二項分布を用いた統計的手法に関連する話題で、通常は成功・失敗の2つの結果が存在する状況を表す。

6. **エピソード (Episode)**:
   強化学習などの文脈で使われる概念。ある一連の行動や試行を指し、特定のタスクを完了するまでの過程を示す。

7. **最大新トークン (max_new_tokens)**:
   モデルが生成する際の最大トークン数。このパラメータによって、生成される応答の長さを制限することができる。

8. **`torch_dtype`**:
   PyTorchと呼ばれる深層学習フレームワークにおいて、テンソルのデータ型を指定する引数。"bfloat16"は、浮動小数点数の一種で、特に深層学習での計算精度と性能のバランスを考慮して使用される。

9. **os.path.exists**:
   Pythonのosモジュールを使ったファイルやディレクトリの存在確認に用いる関数。指定されたパスが存在するかをチェックすることができる。

10. **assert**:
    Pythonのキーワードで、条件が真であることを確認するために使用される。条件が偽の場合には、エラーを引き起こしてプログラムを終了させる。

11. **ヒント (Hint)**:
    ゲームや問題を解決するための手がかり。特にこのコンペティションの文脈では、ユーザーが単語を推測する際に利用する際のアシストを指す。

12. **tar.gz**:
    複数のファイルをまとめるために使用される圧縮ファイル形式の一つ。tarはアーカイブ形式、gzはgzip圧縮を示す。

これらは、初心者が理解する上で役立つ情報を提供し、機械学習や深層学習のコンテキストでも重要な概念です。

---


# このイントロダクションは初心者向けです
## このコンペティションで必要なことは？

**要約：20の質問ゲームをプレイできるモデルを作成すること**

ロボットが話し、歩き、踊ることができるように構築する必要があると想像してみてください。あなたの仕事は、そのロボットが各役割を適切に行い、ユーザーが好きな役割を選択して他の役割に干渉することなく実行できるようにすることです（例えば、3つのボタンを追加できます）。

これはまさに、このコンペティションであなたに求められていることです。質問者、回答者、推測者の3つの役割を果たすことができるエージェント/ロボット（llmモデル）を構築する必要があります。

こちらの概要セクションを読んだ場合、提出したエージェントは、別の参加者が提出したエージェントと対戦することになります。このデュオの中で、あなたのエージェントは質問者と推測者の役割を果たすか、回答者の役割を果たします。役割とバイノーム（2人組）は、裏での環境によっていくつかの条件に基づいて選択され、あなたはそれについて心配する必要はありません。

あなたがすべきことは、環境が回答者の役割をプレイすることを決定したとき、あなたのエージェントが「はい」または「いいえ」で答えることを確認することだけです。他の回答をすることは、ゲームに負けることになります。

詳細はこちらで確認できます [こちら](https://www.kaggle.com/competitions/llm-20-questions/overview) 

## 提出方法は？

コードはmain.pyという1つのファイルにまとめなければならず、このファイルにはエージェント/ロボットのコードと、obsおよびcfgを引数とする必須の関数が含まれている必要があります。これは、裏で環境があなたのエージェントを実行するために使用されます。

私たちのコード（他のノートブックに比べて非常にシンプルです）では、この関数に「agent」という名前を付け、他のロジックを「robot」という名前のクラスに入れます。

環境はオフラインモードでコードを実行し、/kaggle/inputディレクトリにアクセスできないため、必要なパッケージとmain.pyファイルを1つのtar.gzファイルにロード/コピーする必要があります。

裏でtar.gzファイルは「/kaggle_simulations/agent/」フォルダの下に展開されるため、私たちは実行環境に応じてパスを調整するコードを追加します（下記を参照）。

コードをローカルでテストすることもできます。ローカルテストについては、以下の「ローカルテスト」セクションを参照してください。


In [ ]:
%%bash
mkdir -p /kaggle/working/submission

In [ ]:
%%writefile -a submission/main.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from kaggle_secrets import UserSecretsClient
import os
import sys
import shutil

# CUDAのメモリ効率を有効にする
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Kaggleのエージェントパスを設定
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")
else:
    model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

# トークナイザーとモデルを初期化
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]

# 回答を生成する関数
def generate_answer(template):
    inp_ids = tokenizer(template, return_tensors="pt").to("cuda")
    out_ids = model.generate(**inp_ids,max_new_tokens=15).squeeze()
    start_gen = inp_ids.input_ids.shape[1]
    out_ids = out_ids[start_gen:]
    if id_eot in out_ids:
        stop = out_ids.tolist().index(id_eot)
        out = tokenizer.decode(out_ids[:stop])
    else:
        out = tokenizer.decode(out_ids)
    return out
    

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "modeはこれらの値のいずれかでなければなりません：asking, answering, guessing"
        if mode == "asking":
            # 質問者の役割を始める
            output = self.asker(obs)
        if mode == "answering":
            # 回答者の役割を始める
            output = self.answerer(obs)
            if "yes" in output.lower():
                output = "yes"
            elif "no" in output.lower():
                output = "no"   
            if ("yes" not in output.lower() and "no" not in output.lower()):
                output = "yes"  # 明確な回答が無かった場合は「yes」にする
        if mode == "guessing":
            # 推測者の役割を始める
            output = self.asker(obs)
        return output
    
    
    def asker(self, obs):
        sys_prompt = """あなたは有能なAIアシスタントであり、20の質問ゲームをプレイするのが非常に得意です。
        ユーザーは単語を考えるつもりです。それは次の3つのカテゴリのいずれかです：
        1. 場所
        2. 人物
        3. 物
        これらのオプションに焦点を当てて、検索範囲を狭めるための賢い質問をしてください。\n"""
    
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """あなたの役割は、彼に20の質問以内で単語を見つけることです。あなたの質問は有効であるためには「はい」または「いいえ」の回答のみでなければなりません。
            ヒントとして、以下にどのように機能すべきかの例があります。キーワードがモロッコであると仮定します：
            例：
            <あなた：それは場所ですか？
            ユーザー：はい
            あなた：ヨーロッパにありますか？
            ユーザー：いいえ
            あなた：アフリカにありますか？
            ユーザー：はい
            あなた：そこで生きている人々の大半は肌が暗いですか？
            ユーザー：いいえ
            ユーザー：mから始まる国名ですか？
            あなた：はい
            あなた：モロッコですか？
            ユーザー：はい。>

            ユーザーが単語を選びました。最初の質問をしてください！
            短く簡潔にし、余計な言葉は使わず、一つの質問だけをしてください！"""
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{ask_prompt}<|eot_id|>"""
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
            if len(obs.questions)>=1:
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                    chat_template += f"{a}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問: {q}\n回答: {a}\n"""
            guess_prompt =  sys_prompt + f"""これまでのゲームの状態は次の通りです:\n{conv}
            会話に基づいて、単語を推測できますか？単語だけを教えてください、余計な言葉はいりません。"""
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{guess_prompt}<|eot_id|>"""
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
                
        output = generate_answer(chat_template)        
        return output
        
        
        
    def answerer(self, obs):
        sys_prompt = f"""あなたは有能なAIアシスタントであり、20の質問ゲームをプレイするのが非常に得意です。
        ユーザーの役割は、あなたに対して最大20の質問を使って単語を推測することです。あなたの回答は「はい」または「いいえ」でなければなりません。その他の回答は無効であり、それによりゲームに負けます。
        ユーザーは常に次の3つのカテゴリのいずれかに属する単語を推測することになります：
        1. 場所
        2. 人物
        3. 物
        ユーザーの質問を理解し、あなたがプレイしているキーワードを把握してください。
        現在、ユーザーが推測する必要がある単語は「{obs.keyword}」で、カテゴリは「{obs.category}」です。
        ヒントとして、以下にどのように機能すべきかの例があります。キーワードがモロッコでカテゴリ「場所」であると仮定します：
        例：
        <ユーザー：それは場所ですか？
        あなた：はい
        ユーザー：ヨーロッパにありますか？
        あなた：いいえ
        ユーザー：アフリカにありますか？
        あなた：はい
        ユーザー：そこで生きている人々の大半は肌が暗いですか？
        あなた：いいえ
        ユーザー：mから始まる国名ですか？
        あなた：はい
        ユーザー：モロッコですか？
        あなた：はい。>"""
        
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{sys_prompt}<|eot_id|>"""
        chat_template += "<|start_header_id|>user<|end_header_id|>\n\n"
        chat_template += f"{obs.questions[0]}<|eot_id|>"
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{a}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                chat_template += f"{q}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        output = generate_answer(chat_template)
        return output
    
    
robot = Robot()


def agent(obs, cfg):
    
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:
        response = "yes"
        
    return response

# ローカルテスト <a id="lc"></a>

ローカルでテストするには、まず上のセルの「%%writefile -a submission/main.py」をコメントアウトし、このセルを実行します。
次に、以下のセルをコメントアウト解除して実行してください。


In [ ]:
# %%time

# from kaggle_environments import make
# env = make("llm_20_questions", debug=True)
# game_output = env.run(agents=[agent, agent, agent, agent])

In [ ]:
# env.render(mode="ipython", width=600, height=500)

# 提出ファイル


In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/input/llama-3/transformers/8b-chat-hf . -C /kaggle/working/submission .

In [ ]:
# tar.gzファイルの中身を確認する

# import tarfile
# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# for file in tar.getmembers():
#     print(file.name)

---

# コメント 

> ## davide
> 
> 素晴らしい仕事です。共有してくれてありがとうございます！
> 
> ただ質問ですが、このコードはkeyword.pyファイルを全く使用していないのですか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > [@davidemariani](https://www.kaggle.com/davidemariani) いいえ
> > 
> > 
> > 


---

> ## dhruvyadav89300
> 
> ノートブックを説明してくれてありがとう。とても役に立ちました。
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > それを聞いて嬉しいです :)
> > 
> > 
> > 


---

> ## Chazzyie
> 
> あなたの作業を共有してくれてありがとう。シンプルで明確です 🤩
> 
> 


---

> ## Matthew S Farmer
> 
> シンプルなノートブックですね。FYI、このコンペティションでは「人物」というカテゴリが削除されました。それはあなたのエージェントがキーワードの検索空間を少しでも保つのに役立つかもしれません。 
> 
> 
> 
> > ## Matthew S Farmer
> > 
> > [https://www.kaggle.com/competitions/llm-20-questions/discussion/512955#2884981](https://www.kaggle.com/competitions/llm-20-questions/discussion/512955#2884981)
> > 
> > 
> > 
> > ## kaoutarTopic著者
> > 
> > おお、ありがとうございます。知らなかったです！
> > 
> > 
> > 


---

> ## BK
> 
> 共有してくれてありがとう！あなたのノートブックが大好きです :)
> 
> 


---

> ## Eslam Mohamed
> 
> 素晴らしい仕事です！あなたの説明は最高です。
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > それを聞いて嬉しいです！
> > 
> > 
> > 


---

> ## 古月方源
> 
> ローカルテストが成功しました
> 
> 
> 


---

> ## 古月方源
> 
> 最新の3.1を成功裏に実行しましたが、提出後にエラーが発生し続けました
> 
> 
> 


---

> ## Sandeep Sharma
> 
> ノートブックをありがとう。しかし、テストしようとすると「1つの位置引数 'cfg' が不足しています」と表示されます。
> 
> 
> 


---

> ## Toshi
> 
> 素晴らしいノートブックです。
> 
> ローカルでテストする場合、どのバージョンのpytorchをインストールすべきですか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > [@mst923](https://www.kaggle.com/mst923) あなたのパソコンでローカルという意味ですか？良いGPUがない限り、動作しないでしょう
> > 
> > 
> > 


---

> ## kothiwsk28
> 
> 共有してくれてありがとう！このノートブックに3つのデータセット（数字認識、タイタニック、ハウスプライス）を保持しておく特別な理由がありますか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > いいえ、削除した方が良いです。ノートブックには、競技にやってくる初心者を助けるために追加しました。
> > 
> > 
> > 


---

> ## Mask
> 
> 役に立つノートブックですね。
> 
> 私はKaggleに新しく、フォークしたりコピーしたりしたときに、実際に提出ボタンが見つかりません！あなたのノートブックに付属している他の競技（数字認識者とタイタニック）があり、そのノートブックでLlamaを使用してすべてのコードをコピーしたとき、モデルパスが正しくないというエラーが表示されましたが、正しいです！
> 
> 何が間違っているのかよくわかりません！
> 
> あなたのノートブックをフォークして、エラーなしに提出する方法を教えてもらえますか？私はここでは新しいので、使い方を知りたいです。
> 
> 提出ボタンがないのは非常に奇妙で、モデルディレクトリが正しく表示されているのに、正しくないと言われています！
> 
> 
> > ## Maoshenli
> > 
> > 私も同じ問題に直面しました。解決しましたか？😄
> > 
> > 
> > 
> > ## jehlum
> > 
> > モデルを右側の入力タブから追加しましたか？
> > 
> > 
> > 
> > ## kaoutarTopic著者
> > 
> > 皆さん、llamaモデルを使用するための条件を受け入れ、アクセスを許可されていることを確認してください。この問題を解決し、ノートブックが正しく実行されれば、右上の3つの点に行き、「コンペティションに提出」をクリックしてください。
> > 
> > 
> > 


---

> ## Lucas
> 
> コードをありがとうございます。提出しましたが、LBで600点しか得られませんでした。750点を獲得するにはどうすれば良いですか？Llama3 8bを微調整しましたか？
> 
> 
> 
> > ## Krens
> > 
> > ゲームを提出したとき、初期スコアは600ポイントでした。スコアは他の参加者のエージェントとのゲームをプレイすることでのみ変動します。さらに、分散は比較的大きく、それほど安定していません。
> > 
> > 
> > 


---

> ## zerojin
> 
> 作業を共有してくれてありがとう。質問があります。使用しているllama3モデルは微調整されていますか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > 私のデータで微調整されていますか？答えはいいえです。Kaggleハブのローカルモデルのパスを使用していることがわかります。
> > 
> > 
> > 


---

> ## tingyu516
> 
> こんにちは。このコードを提出して「Validation Episode failed」というエラーメッセージを受け取りました。どうすれば良いですか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > エージェントのログをダウンロードして、何が起こったのかを確認できます。
> > 
> > 
> > 


---

> ## Matthew S Farmer
> 
> このコードは競技のGPU上でメモリ不足にならずに実行されるのはなぜですか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > そうですね、私も心配でしたが、うまくいきました。
> > 
> > 
> > 


---

> ## i_am_nothing
> 
> データソースである数字認識者を削除するにはどうすればよいですか？
> 
> 
> > ## Chazzyie
> > 
> > 3つの点をクリックして、数字認識者フォルダーを削除を選択できます。
> > 
> > 
> > 


---

> ## Eslam Mohamed
> 
> 非常に優れたプレゼンテーションです！明確さと詳細のレベルは最高です。
> 
> 


---

> ## philipha2
> 
> 私はこの競技に参加したばかりの初心者です。
> 
> 質問が基本すぎる場合はごめんなさい。
> 
> このコードをcolabで実行しようとしていますが、モジュールや提出コードのために実行できません。このコードはKaggleプラットフォームでしか実行できませんか？
> 
> 
> 
> > ## kaoutarTopic著者
> > 
> > こんにちは、いいえ、Google Colabでは実行できません。モデルのローカルパスを使用しているため、Google Colabを使用する場合はHuggingfaceからモデルをダウンロードし、Kaggle環境パッケージをインストールしてください。
> > 
> > 
> > 
> > > ## philipha2
> > > 
> > > 返信どうもありがとうございます！
> > > 
> > > 通常、あなたはどのようにモデルを実行しますか？
> > > 
> > > Kaggleのリソース（GPU）を使用しているのですか？それとも自分のGPUを使用しているのですか？
> > > 
> > > 
> > > ## kaoutarTopic著者
> > > 
> > > 私は個人的なGPUを持っていません。
> > > 
> > > 
> > > > ## Krens
> > > > 
> > > > > 
> > > > Kaggleのノートブック内の最初のセルのコードを実行すると、Kaggleノートブックの入力ファイルを自動的にダウンロードして正しいファイルパスを作成します。
> > > > 
> > > > 
---